In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [59]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [60]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [61]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df



# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [62]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,twinkling,swallow,gradually,hot,visit,queens,income,fortnight,enable,disgrace,...,supple,economy,trouble,agitation,continue,harden,curly,worm,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


## Trying out BoW

Now let's give the bag of words features a whirl by trying a random forest.

In [63]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts['text_source']
X = np.array(word_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)
train = rfc.fit(X_train, y_train)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.989028213166

Test set score: 0.891917293233


Holy overfitting, Batman! Overfitting is a known problem when using bag of words, since it basically involves throwing a massive number of features at a model – some of those features (in this case, word frequencies) will capture noise in the training set. Since overfitting is also a known problem with Random Forests, the divergence between training score and test score is expected.


## BoW with Logistic Regression

Let's try a technique with some protection against overfitting due to extraneous features – logistic regression with ridge regularization (from ridge regression, also called L2 regularization).

In [8]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3062) (3190,)
Training set score: 0.957993730408

Test set score: 0.915883458647


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [9]:
clf = ensemble.GradientBoostingClassifier()
train = clf.fit(X_train, y_train)

print('Training set score:', clf.score(X_train, y_train))
print('\nTest set score:', clf.score(X_test, y_test))

Training set score: 0.884639498433

Test set score: 0.873590225564


Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

In [10]:
# Clean the Emma data.
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [11]:
# Parse our cleaned data.
emma_doc = nlp(emma)

In [12]:
# Group into sentences.
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [13]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences, common_words)

print('done')

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
done


In [42]:
emma_bow.head()

,twinkling,swallow,gradually,hot,visit,queens,income,fortnight,enable,disgrace,...,supple,economy,trouble,agitation,continue,harden,curly,worm,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(She, was, the, youngest, of, the, two, daught...",Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Her, mother, had, died, too, long, ago, for, ...",Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Between, _, them)",Austen


In [14]:
# Now we can model it!
# Let's use logistic regression again.

# Combine the Emma sentence data with the Alice data from the test set.
X_Emma_test = np.concatenate((
    X_train[y_train[y_train=='Carroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)
y_Emma_test = pd.concat([y_train[y_train=='Carroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])

# Model.
print('\nTest set score:', lr.score(X_Emma_test, y_Emma_test))
lr_Emma_predicted = lr.predict(X_Emma_test)
pd.crosstab(y_Emma_test, lr_Emma_predicted)


Test set score: 0.697613721104


col_0,Austen,Carroll
row_0,,
Austen,1564,105
Carroll,706,307


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

In [15]:
def parts_of_speech(text):
    
    partsofspeech = [text[i].pos_
                     for i in range(0, len(text))]
    
    return [item for item in Counter(partsofspeech)]

In [16]:
# Set up the list of POS.
alicepos = parts_of_speech(alice_doc)
persuasionpos = parts_of_speech(persuasion_doc)

In [17]:
# Combine list to create a set of unique POS.
common_pos = set(alicepos + persuasionpos)

In [18]:
# Create data frame of Parts of Speech
def pos_features(sentences, common_pos):
    
    # Scaffold the data frame and initialize counts to zero.
    df_pos = pd.DataFrame(columns=common_pos)
    df_pos['text_sentence'] = sentences[0]
    df_pos['text_source'] = sentences[1]
    df_pos.loc[:, common_pos] = 0
    
    # Process each row, counting the occurrence of parts of speech in each sentence.
    for i, sentence in enumerate(df_pos['text_sentence']):
        
        # Get part of speech for each word in the sentence
        pos = [sentence[i].pos_
                 for i in range(0, len(sentence))]
        
        
        # Populate the row with parts of speech counts.
        for part in pos:
            df_pos.loc[i, part] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df_pos

In [19]:
# Create our data frame with features. This can take a while to run.
pos_counts = pos_features(sentences, common_pos)
pos_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500
Processing row 4000
Processing row 4500
Processing row 5000


,DET,VERB,ADV,INTJ,NOUN,NUM,ADJ,ADP,PRON,PART,PUNCT,CCONJ,PROPN,X,text_sentence,text_source
0,5,13,3,0,12,0,3,8,3,2,10,6,2,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,6,11,7,0,8,0,7,8,4,1,7,2,2,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,3,5,6,2,2,0,1,4,2,1,4,1,2,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,2,0,0,0,0,0,0,1,0,0,0,"(Oh, dear, !)",Carroll
4,0,2,0,0,0,0,1,0,1,0,2,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [20]:
combined_counts = pd.merge(word_counts, pos_counts, on=['text_sentence','text_source'])

In [21]:
combined_counts.head()

,twinkling,swallow,gradually,hot,visit,queens,income,fortnight,enable,disgrace,...,NOUN,NUM,ADJ,ADP,PRON,PART,PUNCT,CCONJ,PROPN,X
0,0,0,0,0,0,0,0,0,0,0,...,12,0,3,8,3,2,10,6,2,0
1,0,0,0,1,0,0,0,0,0,0,...,8,0,7,8,4,1,7,2,2,0
2,0,0,0,0,0,0,0,0,0,0,...,2,0,1,4,2,1,4,1,2,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,2,0,0,0


In [22]:
Y = combined_counts['text_source']
X = np.array(combined_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [24]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

predict_test = rfc.predict(X_test)

print('Training set score:', rfc.score(X_train, y_train))
print('\nTest set score:', rfc.score(X_test, y_test))

Training set score: 0.986206896552

Test set score: 0.870770676692


In [25]:
# Was able to improve the test score of the Logistic Regression model by a little bit by adding the occurence of the
# parts of speech in the features

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(3190, 3076) (3190,)
Training set score: 0.958934169279

Test set score: 0.916823308271


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

In [26]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [89]:
# Clean the Sense data.
sense = gutenberg.raw('austen-sense.txt')
sense = re.sub(r'VOLUME \w+', '', sense)
sense = re.sub(r'CHAPTER \w+', '', sense)
sense = text_cleaner(sense)
print(sense[:100])

The family of Dashwood had long been settled in Sussex. Their estate was large, and their residence 


In [93]:
# Clean the Macbeth data.
macbeth = gutenberg.raw('shakespeare-macbeth.txt')
macbeth = re.sub(r'VOLUME \w+', '', macbeth)
macbeth = re.sub(r'CHAPTER \w+', '', macbeth)
macbeth = text_cleaner(macbeth)
print(macbeth[:100])

Actus Primus. Scoena Prima. Thunder and Lightning. Enter three Witches. 1. When shall we three meet 


In [91]:
# Parse our cleaned data for Sense.
sense_doc = nlp(sense)

In [94]:
# Parse our cleaned data for Moby Dick.
macbeth_doc = nlp(macbeth)

In [95]:
# Group into sentences.
macbeth_sents = [[sent, "Shakespeare"] for sent in macbeth_doc.sents]
sense_sents = [[sent, "Austen"] for sent in sense_doc.sents]

In [97]:
len(macbeth_sents)

1907

In [98]:
len(sense_sents)

5219

In [99]:
# Sense is quite long, let's cut it down to the same length as Macbeth.
sense_sents = sense_sents[0:len(macbeth_sents)]

In [100]:
# Set up the bags.
macbethwords = bag_of_words(macbeth_doc)
sensewords = bag_of_words(sense_doc)

In [101]:
# Combine bags to create a set of unique words.
common_words = set(macbethwords + sensewords)

In [102]:
# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(macbeth_sents + sense_sents)
sentences.head()

,0,1
0,"(Actus, Primus, .)",Shakespeare
1,"(Scoena, Prima, .)",Shakespeare
2,"(Thunder, and, Lightning, .)",Shakespeare
3,"(Enter, three, Witches, .)",Shakespeare
4,"(1, .)",Shakespeare


In [103]:
# Create our data frame with features. This can take a while to run.
word_counts = bow_features(sentences, common_words)
word_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500


,designe,income,visit,disgrace,fortnight,enable,misapprehension,provision,cut,prompt,...,mercifull,agitation,outlive,continue,bond,males,harden,redouble,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Actus, Primus, .)",Shakespeare
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Scoena, Prima, .)",Shakespeare
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Thunder, and, Lightning, .)",Shakespeare
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Enter, three, Witches, .)",Shakespeare
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(1, .)",Shakespeare


In [104]:
# Set up the list of POS.
macbethpos = parts_of_speech(macbeth_doc)
sensepos = parts_of_speech(sense_doc)

In [105]:
# Combine list to create a set of unique POS.
common_pos = set(macbethpos + sensepos)

In [106]:
# Create our data frame with features. This can take a while to run.
pos_counts = pos_features(sentences, common_pos)
pos_counts.head()

Processing row 0
Processing row 500
Processing row 1000
Processing row 1500
Processing row 2000
Processing row 2500
Processing row 3000
Processing row 3500


,DET,VERB,ADV,INTJ,NOUN,NUM,ADJ,ADP,PRON,PART,PUNCT,CCONJ,PROPN,X,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,1,0,2,0,"(Actus, Primus, .)",Shakespeare
1,0,0,0,0,0,0,0,0,0,0,1,0,2,0,"(Scoena, Prima, .)",Shakespeare
2,0,0,0,0,2,0,0,0,0,0,1,1,0,0,"(Thunder, and, Lightning, .)",Shakespeare
3,0,1,0,0,0,1,0,0,0,0,1,0,1,0,"(Enter, three, Witches, .)",Shakespeare
4,0,0,0,0,0,1,0,0,0,0,1,0,0,0,"(1, .)",Shakespeare


In [107]:
combined_counts = pd.merge(word_counts, pos_counts, on=['text_sentence','text_source'])
combined_counts.head()

,designe,income,visit,disgrace,fortnight,enable,misapprehension,provision,cut,prompt,...,NOUN,NUM,ADJ,ADP,PRON,PART,PUNCT,CCONJ,PROPN,X
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2,0
2,0,0,0,0,0,0,0,0,0,0,...,2,0,0,0,0,0,1,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,0


In [108]:
Y = combined_counts['text_source']
X = np.array(combined_counts.drop(['text_sentence','text_source'], 1))

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.4,
                                                    random_state=0)

In [109]:
lr = LogisticRegression()
train = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print('Training set score:', lr.score(X_train, y_train))
print('\nTest set score:', lr.score(X_test, y_test))

(2288, 3498) (2288,)
Training set score: 0.975524475524

Test set score: 0.931847968545
